In [1]:
from Recommendation.recommendation_utils import get_ai_client_requirements

get_ai_client_requirements("Growth sutra is a LLP company knwon for GTM strategies","lient want to increase his sales by 10%")

"---\n### ✅ Refined Client Requirements\n\n**Project Title:** Growth Acceleration Program for [Client Name]\n\n**Project Goal:** To achieve a minimum 10% year-over-year increase in sales revenue within the next 12 months.  This will be measured by comparing Q4 2024 revenue against Q4 2023 revenue.\n\n**Project Scope:** This project will encompass a comprehensive review of [Client Name]'s current Go-to-Market (GTM) strategy, including but not limited to: market analysis, competitive landscape assessment, sales process optimization, lead generation strategies, sales enablement, and marketing alignment.  Growth Sutra will leverage its expertise in GTM strategy development and implementation to deliver tailored solutions.\n\n**Deliverables:**\n\n* **Phase 1 (Months 1-3):**  GTM Strategy Audit & Roadmap Development –  Comprehensive assessment of existing GTM processes, identification of key areas for improvement, and development of a detailed roadmap with prioritized initiatives.  Deliverab

In [1]:
from Recommendation.recommendation_utils import get_ai_business_priorities

result = get_ai_business_priorities("CEO")

/home/shreyank/anaconda3/envs/adk/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[{'title': 'Sustainable Revenue Growth & Market Share Expansion', 'icon': '📈'}, {'title': 'Digital Transformation & Operational Excellence', 'icon': '⚙️'}, {'title': 'Talent Acquisition & Retention in a Competitive Market', 'icon': '🧑\u200d💼'}]


In [2]:
type(result)

list

In [ ]:

import requests
from dotenv import load_dotenv
load_dotenv()
import os
import streamlit as st


def search_linkedin_serpapi(name):
    params = {
        "api_key": os.getenv("SERP_API_KEY"),
        "engine": "google",
        "num": 5
    }
    
    try:
        response = requests.get("https://serpapi.com/search", params=params).json()
        results = {}
        
        for res in response.get("organic_results", []):
            results.append(res)
        
        print(results)
        return results
    except Exception as e:
        st.error(f"Error searching LinkedIn profiles: {e}")
        return {}
